In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00


In [5]:
import os
import glob
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

# Đường dẫn tới thư mục chứa dữ liệu huấn luyện và dữ liệu test
BASE_DIR = '/content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TRAIN'
TEST_DIR = '/content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST'
RESULT_DIR = '/content/drive/MyDrive/PPSKHDL_NHOM_/RESULT'

# Hàm đọc danh sách stopwords từ file
def read_stopwords(path):
    with open(path, 'r', encoding='utf-8') as file:
        return set(file.read().splitlines())

# Hàm tách từ trong văn bản tiếng Việt sử dụng ViTokenizer
def segmentation(text):
    from pyvi import ViTokenizer
    return ViTokenizer.tokenize(text)

# Hàm chia văn bản thành các từ
def split_words(text):
    text = segmentation(text)
    return text.split()

# Hàm loại bỏ các ký tự đặc biệt khỏi từ
def remove_special_chars(word, special_chars):
    return ''.join(char for char in word if char not in special_chars)

# Hàm làm sạch văn bản bằng cách tách từ, loại bỏ ký tự đặc biệt và stopwords
def get_clean_text(text, stopwords, special_chars):
    words = split_words(text)
    return ' '.join([remove_special_chars(word.lower(), special_chars) for word in words if word.lower() not in stopwords])

# Hàm đọc tệp tin với nhiều mã hóa khác nhau
def read_file(file_path, encodings=['utf-8', 'latin-1', 'cp1252']):
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to read file {file_path} with given encodings.")

# Hàm đọc dữ liệu từ thư mục và tiền xử lý văn bản
def read_data_from_directory(base_dir):
    texts = []
    labels = []
    for category in os.listdir(base_dir):
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            for file_path in glob.glob(os.path.join(category_path, '*.txt')):
                text = read_file(file_path)
                texts.append(get_clean_text(text, stopwords, special_chars))
                labels.append(category)
    return texts, labels

# Hàm phân loại và sao chép các file vào thư mục tương ứng với nhãn dự đoán
def classify_and_copy_files(test_dir, result_dir, pipeline, label_encoder):
    for file_path in glob.glob(os.path.join(test_dir, '*.txt')):
        text = read_file(file_path)
        if text.strip():
            test_text_clean = get_clean_text(text, stopwords, special_chars)
            prediction = pipeline.predict([test_text_clean])
            predicted_label = label_encoder.inverse_transform(prediction)[0]

            destination_dir = os.path.join(result_dir, predicted_label)
            os.makedirs(destination_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
            shutil.copy(file_path, destination_dir)  # Sao chép file vào thư mục tương ứng
            print(f"Copied {file_path} to {destination_dir}")
        else:
            print(f"Error: Empty test file {file_path}.")

# Đọc danh sách stopwords và ký tự đặc biệt
stopwords = read_stopwords('/content/drive/MyDrive/PPSKHDL_NHOM_/stopwords_VIE.txt')
special_chars = set(".,!?\"':;()")

# Đọc và tiền xử lý dữ liệu huấn luyện
train_texts, train_labels = read_data_from_directory(BASE_DIR)
print("Unique labels:", set(train_labels))
print("Number of unique labels:", len(set(train_labels)))

# Kiểm tra số lượng nhãn phải lớn hơn 1
if len(set(train_labels)) <= 1:
    print("Error: The number of classes has to be greater than one.")
else:
    # Mã hóa nhãn thành các số nguyên
    label_encoder = LabelEncoder()
    train_labels_encoded = label_encoder.fit_transform(train_labels)

    # Tạo pipeline với TfidfVectorizer, TruncatedSVD và SVC
    pipeline = make_pipeline(
        TfidfVectorizer(),
        TruncatedSVD(n_components=100),
        SVC()
    )
    # Huấn luyện mô hình với dữ liệu huấn luyện
    pipeline.fit(train_texts, train_labels_encoded)

    # Phân loại và sao chép các file test vào thư mục tương ứng
    classify_and_copy_files(TEST_DIR, RESULT_DIR, pipeline, label_encoder)


Unique labels: {'GIAO DUC', 'XA HOI', 'VAN HOA', 'THE THAO', 'PHAP LUAT', 'KINH TE', 'Y TE'}
Number of unique labels: 7
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST/TE0001.txt to /content/drive/MyDrive/PPSKHDL_NHOM_/RESULT/XA HOI
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST/TE0002.txt to /content/drive/MyDrive/PPSKHDL_NHOM_/RESULT/XA HOI
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST/TE0003.txt to /content/drive/MyDrive/PPSKHDL_NHOM_/RESULT/XA HOI
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST/TE0004.txt to /content/drive/MyDrive/PPSKHDL_NHOM_/RESULT/XA HOI
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST/TE0005.txt to /content/drive/MyDrive/PPSKHDL_NHOM_/RESULT/XA HOI
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST/TE0006.txt to /content/drive/MyDrive/PPSKHDL_NHOM_/RESULT/KINH TE
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/DATA_TEST/TE0007.txt to /content/drive/MyDrive/PPSKHDL_NHOM_/RESULT/KINH TE
Copied /content/drive/MyDrive/PPSKHDL_NHOM_/D

In [9]:
import os

def count_files_in_directory(result_dir):
    counts = {}
    for root, dirs, files in os.walk(result_dir):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            file_count = len([file for file in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, file))])
            counts[dir_name] = file_count
    return counts

def print_counts(counts):
    print(f"{'Directory':<20}{'File Count':<10}")
    print("=" * 30)
    for dir_name, file_count in counts.items():
        print(f"{dir_name:<20}{file_count:<10}")

if __name__ == "__main__":
    RESULT_DIR = '/content/drive/MyDrive/PPSKHDL_NHOM_/RESULT'

    # Count files in each subdirectory of RESULT_DIR
    file_counts = count_files_in_directory(RESULT_DIR)

    # Print the counts
    print_counts(file_counts)


Directory           File Count
Y TE                209       
KINH TE             55        
GIAO DUC            15        
XA HOI              334       
THE THAO            129       
PHAP LUAT           107       
VAN HOA             15        


# Mục mới